In [6]:
import json

In [7]:
#open url information
fd = open('./data/project_successful_meta.json', 'r')
text = fd.read()
fd.close()

pmeta = json.loads(text)

In [32]:
class Doc():
    
    def __init__(self, docid, doc, author, year):
        self.docid = docid
        self.text = doc.lower()
        self.text = re.sub(u'[\u2019\']', '', self.text)
        self.tokens = np.array(wordpunct_tokenize(self.text))
        self.stem = None
        self.author = author
        self.year = year
        
    def tf(self, wordlist):
        
        """
        Returns ARRAY woth wordlist frequency
        """
        
        count = np.zeros(len(wordlist))
        
        for wid, word in np.ndenumerate(wordlist):
            count[wid] = (self.tokens == word).sum()
        return count
        
    
    def word_exists(self, wordlist):
        
        """
        Return dict with log count of wordlist
        """
        
        is_word = np.zeros(len(wordlist))
        
        for wid, word in np.ndenumerate(wordlist):
            if word in self.tokens:
                is_word[wid] = 1
        return is_word
            
    def token_clean(self,length):

        """ 
        strip out non-alpha tokens and length one tokens
        """

        self.tokens = np.array([t for t in self.tokens if (t.isalpha() and len(t) > length)])


    def stopword_remove(self, stopwords):

        """
        Remove stopwords from tokens.
        """

        
        self.tokens = np.array([t for t in self.tokens if t not in stopwords])


    def stem(self):

        """
        Stem tokens with Porter Stemmer.
        """
        
        self.stems = n.array([PorterStemmer().stem(t) for t in self.tokens])

In [2]:
import codecs,re
from nltk.tokenize import wordpunct_tokenize
from nltk import PorterStemmer

In [85]:
from math import log
import numpy as np
class RawDocs():
    
    def __init__(self, doc_data, stopword_file):

        self.docs = [Doc(docid, doc[2], doc[1], doc[0]) for docid, doc in enumerate(doc_data)]

        with codecs.open(stopword_file,'r','utf-8') as f: raw = f.read()
        self.stopwords = set(raw.splitlines())

        self.N = len(self.docs)
        
    def clean_docs(self, length):
        
        for doc in self.docs:
            doc.token_clean(length)
            doc.stopword_remove(self.stopwords)
            
    def count(self, dictionary):
        
        return ({doc.docid : doc.tf(dictionary) for doc in self.docs})
    
    def idf(self, dictionary):
        
        is_word_docs = {doc.docid : doc.word_exists(dictionary) for doc in self.docs}
        
        return(np.log(self.N / sum([is_word for is_word in is_word_docs.values()])))
    
    def tf_idf(self, dictionary):
        
        tf = self.count(dictionary)
        idf = self.idf(dictionary)
        
        td_idf_docs = dict()
        #convert counts t log counts
        for did in range(self.N):
            td_idf_docs[did] = np.log(tf[did] + 1) * idf
            
        return(td_idf_docs)

In [13]:
import json
fd = open('./data/pres_speech.json', 'r')
text = fd.read()
fd.close()

pres_speech = json.loads(text)

In [86]:
speech_docs = RawDocs(pres_speech[0: 10], './data/stopwords.txt')
speech_docs.clean_docs(2)
doc1 = speech_docs.docs[0]

In [87]:
speech_docs.count(['god', 'bless', 'america', 'race', 'human'])

{0: array([  3.,   2.,  24.,   3.,   2.]),
 1: array([  3.,   3.,  33.,   5.,   1.]),
 2: array([  2.,   2.,  18.,   0.,   1.]),
 3: array([  2.,   2.,  18.,   0.,   2.]),
 4: array([  2.,   2.,  18.,   9.,   0.]),
 5: array([  2.,   2.,  30.,   1.,   2.]),
 6: array([  1.,   1.,  15.,   0.,   5.]),
 7: array([  2.,   1.,  38.,   0.,   7.]),
 8: array([  2.,   1.,  24.,   0.,   2.]),
 9: array([  1.,   1.,  30.,   0.,   2.])}

In [81]:
print(speech_docs.idf(['god', 'bless', 'america', 'race', 'human']))

[ 0.          0.          0.          0.91629073  0.10536052]


In [88]:
speech_docs.tf_idf(['god', 'bless', 'america', 'race', 'human'])

{0: array([ 0.        ,  0.        ,  0.        ,  1.27024867,  0.11575036]),
 1: array([ 0.        ,  0.        ,  0.        ,  1.6417726 ,  0.07303034]),
 2: array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.07303034]),
 3: array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.11575036]),
 4: array([ 0.        ,  0.        ,  0.        ,  2.10983738,  0.        ]),
 5: array([ 0.        ,  0.        ,  0.        ,  0.63512434,  0.11575036]),
 6: array([ 0.       ,  0.       ,  0.       ,  0.       ,  0.1887807]),
 7: array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.21909103]),
 8: array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.11575036]),
 9: array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.11575036])}

In [67]:
import numpy as np
a = np.array([1,2,3])
b = np.ones(3)
c = [a, b]
d = sum(c)

a * b

array([ 1.,  2.,  3.])